In [55]:
import pandas as pd
import geohash

In [65]:
populated_places_csv = 'ne_110m_populated_places.csv'

df = pd.read_csv(populated_places_csv)
REGION_HEADER = 'ADM0_A3'
LAT_HEADER = 'LATITUDE'
LON_HEADER = 'LONGITUDE'
NAME_HEADER = 'NAMEASCII'
POPULATION_HEADER = 'POP_MAX'
TIMEZONE_HEADER = 'TIMEZONE'
GEOHASH_HEADER = 'geohash' # generated

export_columns = [NAME_HEADER, REGION_HEADER, GEOHASH_HEADER, POPULATION_HEADER, TIMEZONE_HEADER]

In [66]:
def encode_geohash(row):
    return geohash.encode(row[LAT_HEADER], row[LON_HEADER], 8)

In [67]:
df['geohash'] = df.apply(encode_geohash, axis=1)

In [68]:
df_export = df[export_columns].rename(columns={
    TIMEZONE_HEADER: "timezone",
    NAME_HEADER: "name",
    POPULATION_HEADER: "population",
    REGION_HEADER: "region",
})

# Clean

In [69]:
df_export[df_export['timezone'].isna()]

,name,region,geohash,population,timezone
9,Bir Lehlou,SAH,eu9hs6g5,500,NaN
147,Sanaa,YEM,sfxrmgsg,2008000,NaN
149,Damascus,SYR,svcy9fz7,2466000,NaN
164,Tripoli,LBY,smc75gvk,2189000,NaN
176,Denver,USA,9xj64fxs,2313000,NaN
177,Houston,USA,9vk1wkyj,4459000,NaN
213,Melbourne,AUS,r1r0fgqj,4170000,NaN
223,Moscow,RUS,ucftpvtr,10452000,NaN
240,Sydney,AUS,r3gx032k,4630000,NaN


In [70]:
patch_timezone = {
    "Bir Lehlou":
    {
        "time_zone": "Africa/El_Aaiun"
    },
    "Sanaa":
    {
        "time_zone": "Asia/Aden"
    },
    "Damascus":
    {
        "time_zone": "Asia/Damascus"
    },
    "Tripoli":
    {
        "time_zone": "Africa/Tripoli"
    },
    "Denver":
    {
        "time_zone": "America/Denver"
    },
    "Houston":
    {
        "time_zone": "America/Chicago"
    },
    "Melbourne":
    {
        "time_zone": "Australia/Melbourne"
    },
    "Moscow":
    {
        "time_zone": "Europe/Moscow"
    },
    "Sydney":
    {
        "time_zone": "Australia/Sydney"
    }
}

In [71]:
for name, d in patch_timezone.items():
    print('patch', name, d['time_zone'])
    df_export.loc[df_export['name'] == name, 'timezone'] = d['time_zone']

patch Bir Lehlou Africa/El_Aaiun
patch Sanaa Asia/Aden
patch Damascus Asia/Damascus
patch Tripoli Africa/Tripoli
patch Denver America/Denver
patch Houston America/Chicago
patch Melbourne Australia/Melbourne
patch Moscow Europe/Moscow
patch Sydney Australia/Sydney


# Modifications/Patch

In [72]:
df_export.loc[df_export['name']=='Singapore', 'geohash'] = 'w21z9vkk' # Prevent singapore underwater

# Export to JSON

In [74]:
df_export['description'] = ''

df_export.to_json('sanctuaries.json', orient='records')

In [ ]:
descriptions_prompt = """
I am creating a MUD, i needed descriptions for the following cities which are quite populated, it is a relaxed style MUD, descriptions should be short and brief but unique. Use the real world knowledge of the culture, heritage, geology, topology of the region. It is a fantasy reimagining the world so do not include any specific details about people or places. keep the descriptions to about 20-30 worlds, make it brief and concise

These are the locations i need, just return me a csv with only 2 columns: name, description
"""